In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns


from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
from keras import regularizers

from keras.datasets import mnist
from keras.utils.np_utils import to_categorical

from keras.preprocessing.image import ImageDataGenerator

import os, shutil

plt.style.use('ggplot')

%matplotlib inline

In [3]:
# importing dataset

original_dataset_dir = 'data/PetImages'

# subset directories to store images

# directory names

base_dir = 'data/cats_and_dogs_small'

train_dir = os.path.join(base_dir,'train')
train_cats_dir = os.path.join(train_dir,'cats')
train_dogs_dir = os.path.join(train_dir,'dogs')

validation_dir = os.path.join(base_dir,'validation')
validation_cats_dir = os.path.join(validation_dir,'cats')
validation_dogs_dir = os.path.join(validation_dir,'dogs')

test_dir = os.path.join(base_dir,'test')
test_cats_dir = os.path.join(test_dir,'cats')
test_dogs_dir = os.path.join(test_dir,'dogs')

try:
    # creating directories: 
    
    os.mkdir(base_dir)
    
    os.mkdir(train_dir)    
    os.mkdir(train_cats_dir)
    os.mkdir(train_dogs_dir)        
    
    os.mkdir(validation_dir)    
    os.mkdir(validation_cats_dir)    
    os.mkdir(validation_dogs_dir)
    
    
    os.mkdir(test_dir)    
    os.mkdir(test_cats_dir)    
    os.mkdir(test_dogs_dir)
    
    for i in range(1000):
        # training sample
        
        file_name = str(i)+'.jpg'
        
        # for cats
        src = os.path.join(original_dataset_dir,'Cat',file_name)
        dst = os.path.join(train_cats_dir,file_name)
        shutil.copyfile(src,dst)
        
        #for dogs
        src = os.path.join(original_dataset_dir,'Dog',file_name)
        dst = os.path.join(train_dogs_dir,file_name)
        shutil.copyfile(src,dst)
        
    for i in range(1000,1500):
        
        # validation sample
        
        file_name = str(i)+'.jpg'
        
        # for cats
        src = os.path.join(original_dataset_dir,'Cat',file_name)
        dst = os.path.join(validation_cats_dir,file_name)
        shutil.copyfile(src,dst)
        
        #for dogs
        src = os.path.join(original_dataset_dir,'Dog',file_name)
        dst = os.path.join(validation_dogs_dir,file_name)
        shutil.copyfile(src,dst)
        
        # testing sample
        
        j = i + 500
        
        file_name = str(j)+'.jpg'
        
        # for cats
        src = os.path.join(original_dataset_dir,'Cat',file_name)
        dst = os.path.join(test_cats_dir,file_name)
        shutil.copyfile(src,dst)
        
        #for dogs
        src = os.path.join(original_dataset_dir,'Dog',file_name)
        dst = os.path.join(test_dogs_dir,file_name)
        shutil.copyfile(src,dst)
        
    
    
except FileExistsError:
    print('base_dir:',base_dir,'already exist')
    print('no sub-directories will be created')

base_dir: data/cats_and_dogs_small already exist
no sub-directories will be created


In [4]:
# check number of images in each sample
print('Total Cat images')
print('training images: ',len(os.listdir(train_cats_dir)))
print('validation images: ',len(os.listdir(validation_cats_dir)))
print('testing images: ', len(os.listdir(test_cats_dir)))
print('')
print('Total Dog images ')
print('training images: ',len(os.listdir(train_dogs_dir)))
print('validation images: ',len(os.listdir(validation_dogs_dir)))
print('testing images: ', len(os.listdir(test_dogs_dir)))

Total Cat images
training images:  1000
validation images:  500
testing images:  500

Total Dog images 
training images:  1000
validation images:  500
testing images:  500


In [5]:
# Instantiating a small convnet


model = models.Sequential()

model.add(layers.Conv2D(filters=32,
                        kernel_size=[3,3],
                        activation='relu',
                        input_shape=[150,150,3]))
model.add(layers.MaxPool2D(pool_size=[2,2]))

model.add(layers.Conv2D(filters=64,
                        kernel_size=[3,3],
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=[2,2]))

model.add(layers.Conv2D(filters=128,
                        kernel_size=[3,3],
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=[2,2]))

model.add(layers.Conv2D(filters=128,
                        kernel_size=[3,3],
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=[2,2]))

model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [6]:
# configure the model for training

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [6]:
# reading images

# rescaling all images by 1/255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=[150,150]
                                                    batch_size=20,
                                                    class_mode='binary') # binary labels
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=[150,150],
                                                        batch_size=20,
                                                        class_mode='binary')

0.0001